In [298]:
# Greenblatt's Magic Formula Implementation

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [299]:
tickers = ["MMM","AXP","AAPL","BA","CAT","CVX","CSCO","KO","DIS","DWDP",
           "XOM","GE","GS","HD","IBM","INTC","JNJ","JPM","MCD","MRK",
           "MSFT","NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT"]

In [300]:
#list of tickers whose financial data needs to be extracted
financial_dir = {}

In [301]:
for ticker in tickers:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"}) 
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2])>0:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
        
        #combining all extracted information with the corresponding ticker
        financial_dir[ticker] = temp_dir
    except:
        print("Problem scraping data for ",ticker)

In [302]:
financial_dir["MSFT"] # Let's check out one of the stocks

{'Assets': 'Current assets',
 'Current assets': 'Cash',
 'Cash': 'Cash and cash equivalents',
 'Cash and cash equivalents': '1,35,76,000',
 'Other short-term investments': '12,29,51,000',
 'Total cash': '13,65,27,000',
 'Net receivables': '3,20,11,000',
 'Inventory': '-6,00,000',
 'Other current assets': '1,14,82,000',
 'Total current assets': '18,19,15,000',
 'Non-current assets': 'Property, plant and equipment',
 'Property, plant and equipment': 'Gross property, plant and equipment',
 'Gross property, plant and equipment': '9,61,01,000',
 'Accumulated depreciation': '-4,31,97,000',
 'Net property, plant and equipment': '5,29,04,000',
 'Equity and other investments': '29,65,000',
 'Goodwill': '4,33,51,000',
 'Intangible assets': '70,38,000',
 'Other long-term assets': '1,31,38,000',
 'Total non-current assets': '11,93,96,000',
 'Total assets': '30,13,11,000',
 "Liabilities and stockholders' equity": 'Liabilities',
 'Liabilities': 'Current liabilities',
 'Current liabilities': 'Current

In [270]:
#storing information in pandas dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials

MMM          AXP  \
Market cap (intra-day)                           120.18B      127.47B   
Enterprise value                                  131.7B       131.5B   
Trailing P/E                                       21.22        26.01   
Forward P/E                                        19.58        17.44   
PEG Ratio (5 yr expected)                           2.58         0.92   
Price/sales                                         3.65         3.82   
Price/book                                          8.73         5.21   
Enterprise value/revenue                            4.00         3.94   
Enterprise value/EBITDA                            14.02          N/A   
Beta (5Y monthly)                                   0.96         1.29   
52-week change                                    39.35%       86.31%   
S&P500 52-week change                             44.45%       44.45%   
52-week high                                      208.95       160.68   
52-week low                                       131.12        76.00   
50-day moving average                             196.76       147.68   
200-day moving average                            179.67       128.56   
Avg vol (3-month)                                  2.38M        3.17M   
Avg vol (10-day)                                   1.87M        2.68M   
Shares outstanding                               579.68M       803.3M   
Implied shares outstanding                           N/A          N/A   
Float                                            579.02M      650.59M   
% held by insiders                                 0.12%        0.12%   
% held by institutions                            67.52%       86.58%   
Shares short (14-Apr-2021)                        10.28M         6.5M   
Short ratio (14-Apr-2021)                            4.2         2.31   
Short % of float (14-Apr-2021)                     1.78%        1.00%   
Short % of shares outstanding (14-Apr-2021)        1.77%        0.81%   
Shares short (prior month 14-Mar-2021)             9.95M        8.67M   
Forward annual dividend rate                        5.92         1.72   
Forward annual dividend yield                      2.92%        1.08%   
Trailing annual dividend rate                       5.89         1.72   
Trailing annual dividend yield                     2.90%        1.08%   
5-year average dividend yield                       2.86         1.49   
Payout ratio                                      60.29%       28.20%   
Dividend date                                11-Mar-2021  09-Aug-2021   
Ex-dividend date                             10-Feb-2021  30-Jun-2021   
Last split factor                                    2:1   10000:8753   
Last split date                              29-Sep-2003  02-Oct-2005   
Fiscal year ends                             30-Dec-2020  30-Dec-2020   
Most-recent quarter                          30-Mar-2021  30-Mar-2021   
Profit margin                                     17.29%       14.98%   
Operating margin                                  22.63%       19.80%   
Return on assets                                   9.98%        2.64%   
Return on equity                                  47.47%       22.01%   
Revenue                                           32.96B       33.41B   
Revenue per share                                  56.97        41.54   
Quarterly revenue growth                           9.60%       26.70%   
Gross profit                                      15.69B       22.09B   
EBITDA                                             9.39B          N/A   
Net income avi to common                            5.7B        4.91B   
Diluted EPS                                         9.77         6.10   
Quarterly earnings growth                         24.20%      509.00%   
Total cash                                         5.14B       39.91B   
Total cash per share                                8.87        49.68   
Total debt                            

In [271]:
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all NaN values
combined_financials

MMM          AXP  \
Market cap (intra-day)                           120.18B      127.47B   
Enterprise value                                  131.7B       131.5B   
Trailing P/E                                       21.22        26.01   
Forward P/E                                        19.58        17.44   
PEG Ratio (5 yr expected)                           2.58         0.92   
Price/sales                                         3.65         3.82   
Price/book                                          8.73         5.21   
Enterprise value/revenue                            4.00         3.94   
Enterprise value/EBITDA                            14.02          N/A   
Beta (5Y monthly)                                   0.96         1.29   
52-week change                                    39.35%       86.31%   
S&P500 52-week change                             44.45%       44.45%   
52-week high                                      208.95       160.68   
52-week low                                       131.12        76.00   
50-day moving average                             196.76       147.68   
200-day moving average                            179.67       128.56   
Avg vol (3-month)                                  2.38M        3.17M   
Avg vol (10-day)                                   1.87M        2.68M   
Shares outstanding                               579.68M       803.3M   
Implied shares outstanding                           N/A          N/A   
Float                                            579.02M      650.59M   
% held by insiders                                 0.12%        0.12%   
% held by institutions                            67.52%       86.58%   
Shares short (14-Apr-2021)                        10.28M         6.5M   
Short ratio (14-Apr-2021)                            4.2         2.31   
Short % of float (14-Apr-2021)                     1.78%        1.00%   
Short % of shares outstanding (14-Apr-2021)        1.77%        0.81%   
Shares short (prior month 14-Mar-2021)             9.95M        8.67M   
Forward annual dividend rate                        5.92         1.72   
Forward annual dividend yield                      2.92%        1.08%   
Trailing annual dividend rate                       5.89         1.72   
Trailing annual dividend yield                     2.90%        1.08%   
5-year average dividend yield                       2.86         1.49   
Payout ratio                                      60.29%       28.20%   
Dividend date                                11-Mar-2021  09-Aug-2021   
Ex-dividend date                             10-Feb-2021  30-Jun-2021   
Last split factor                                    2:1   10000:8753   
Last split date                              29-Sep-2003  02-Oct-2005   
Fiscal year ends                             30-Dec-2020  30-Dec-2020   
Most-recent quarter                          30-Mar-2021  30-Mar-2021   
Profit margin                                     17.29%       14.98%   
Operating margin                                  22.63%       19.80%   
Return on assets                                   9.98%        2.64%   
Return on equity                                  47.47%       22.01%   
Revenue                                           32.96B       33.41B   
Revenue per share                                  56.97        41.54   
Quarterly revenue growth                           9.60%       26.70%   
Gross profit                                      15.69B       22.09B   
EBITDA                                             9.39B          N/A   
Net income avi to common                            5.7B        4.91B   
Diluted EPS                                         9.77         6.10   
Quarterly earnings growth                         24.20%      509.00%   
Total cash                                         5.14B       39.91B   
Total cash per share                                8.87        49.68   
Total debt                            

In [272]:
# Updating the tickers list based on only those tickers whose values were successfully extracted
tickers = combined_financials.columns 
tickers

Index(['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'XOM',
       'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MSFT', 'NKE', 'PFE',
       'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT'],
      dtype='object')

In [273]:
cf_index =combined_financials.index
cf_index

Index(['Market cap (intra-day)', 'Enterprise value', 'Trailing P/E',
       'Forward P/E', 'PEG Ratio (5 yr expected)', 'Price/sales', 'Price/book',
       'Enterprise value/revenue', 'Enterprise value/EBITDA',
       'Beta (5Y monthly)', '52-week change', 'S&P500 52-week change',
       '52-week high', '52-week low', '50-day moving average',
       '200-day moving average', 'Avg vol (3-month)', 'Avg vol (10-day)',
       'Shares outstanding', 'Implied shares outstanding', 'Float',
       '% held by insiders', '% held by institutions',
       'Shares short (14-Apr-2021)', 'Short ratio (14-Apr-2021)',
       'Short % of float (14-Apr-2021)',
       'Short % of shares outstanding (14-Apr-2021)',
       'Shares short (prior month 14-Mar-2021)',
       'Forward annual dividend rate', 'Forward annual dividend yield',
       'Trailing annual dividend rate', 'Trailing annual dividend yield',
       '5-year average dividend yield', 'Payout ratio', 'Dividend date',
       'Ex-dividend date', 'L

In [274]:
# Series.str.contains() function to find if a pattern is contained in the string present in the underlying data of the given series object.

# Creating the Series
sr = pd.Series(['New_York', 'Lisbon', 56, 'Paris', 'Munich'])
  
# Creating the index
idx = ['City 1', 'City 2', 'City 3', 'City 4', 'City 5']

# set the index
sr.index = idx
  
# Print the series
print(sr)

# find if a pattern is contained in the string 
result = sr.str.contains(pat = '[a-z]')
  
# print the result
print(result)

City 1    New_York
City 2      Lisbon
City 3          56
City 4       Paris
City 5      Munich
dtype: object
City 1    True
City 2    True
City 3     NaN
City 4    True
City 5    True
dtype: object


In [275]:
combined_financials[ticker].str.contains("[a-z]")

Market cap (intra-day)                         False
Enterprise value                               False
Trailing P/E                                   False
Forward P/E                                    False
PEG Ratio (5 yr expected)                      False
Price/sales                                    False
Price/book                                     False
Enterprise value/revenue                       False
Enterprise value/EBITDA                        False
Beta (5Y monthly)                              False
52-week change                                 False
S&P500 52-week change                          False
52-week high                                   False
52-week low                                    False
50-day moving average                          False
200-day moving average                         False
Avg vol (3-month)                              False
Avg vol (10-day)                               False
Shares outstanding                            

In [276]:
combined_financials[ticker].str.contains("[a-z]").fillna(False)

Market cap (intra-day)                         False
Enterprise value                               False
Trailing P/E                                   False
Forward P/E                                    False
PEG Ratio (5 yr expected)                      False
Price/sales                                    False
Price/book                                     False
Enterprise value/revenue                       False
Enterprise value/EBITDA                        False
Beta (5Y monthly)                              False
52-week change                                 False
S&P500 52-week change                          False
52-week high                                   False
52-week low                                    False
50-day moving average                          False
200-day moving average                         False
Avg vol (3-month)                              False
Avg vol (10-day)                               False
Shares outstanding                            

In [277]:
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]
    
combined_financials

MMM         AXP     AAPL  \
Market cap (intra-day)                       120.18B     127.47B    2.12T   
Enterprise value                              131.7B      131.5B    2.22T   
Trailing P/E                                   21.22       26.01    28.51   
Forward P/E                                    19.58       17.44    23.75   
PEG Ratio (5 yr expected)                       2.58        0.92     1.45   
Price/sales                                     3.65        3.82     6.51   
Price/book                                      8.73        5.21    30.60   
Enterprise value/revenue                        4.00        3.94     6.84   
Enterprise value/EBITDA                        14.02         N/A    22.29   
Beta (5Y monthly)                               0.96        1.29     1.20   
52-week change                                39.35%      86.31%   65.34%   
S&P500 52-week change                         44.45%      44.45%   44.45%   
52-week high                                  208.95      160.68   145.09   
52-week low                                   131.12       76.00    75.05   
50-day moving average                         196.76      147.68   129.05   
200-day moving average                        179.67      128.56   126.33   
Avg vol (3-month)                              2.38M       3.17M   102.3M   
Avg vol (10-day)                               1.87M       2.68M   93.91M   
Shares outstanding                           579.68M      803.3M   16.69B   
Implied shares outstanding                       N/A         N/A      N/A   
Float                                        579.02M     650.59M   16.67B   
% held by insiders                             0.12%       0.12%    0.08%   
% held by institutions                        67.52%      86.58%   60.01%   
Shares short (14-Apr-2021)                    10.28M        6.5M  127.94M   
Short ratio (14-Apr-2021)                        4.2        2.31     1.29   
Short % of float (14-Apr-2021)                 1.78%       1.00%    0.77%   
Short % of shares outstanding (14-Apr-2021)    1.77%       0.81%    0.77%   
Shares short (prior month 14-Mar-2021)         9.95M       8.67M  107.01M   
Forward annual dividend rate                    5.92        1.72     0.88   
Forward annual dividend yield                  2.92%       1.08%    0.68%   
Trailing annual dividend rate                   5.89        1.72     0.82   
Trailing annual dividend yield                 2.90%       1.08%    0.63%   
5-year average dividend yield                   2.86        1.49     1.37   
Payout ratio                                  60.29%      28.20%   18.34%   
Last split factor                                2:1  10000:8753      4:1   
Profit margin                                 17.29%      14.98%   23.45%   
Operating margin                              22.63%      19.80%   27.32%   
Return on assets                               9.98%       2.64%   16.90%   
Return on equity                              47.47%      22.01%  103.40%   
Revenue                                       32.96B      33.41B  325.41B   
Revenue per share                              56.97       41.54    19.14   
Quarterly revenue growth                       9.60%      26.70%   53.60%   
Gross profit                                  15.69B      22.09B  104.96B   
EBITDA                                         9.39B         N/A   99.82B   
Net income avi to common                        5.7B       4.91B   76.31B   
Diluted EPS                                     9.77        6.10     4.45   
Quarterly earnings growth                     24.20%     509.00%  110.10%   
Total cash                                     5.14B      39.91B   69.83B   
Total cash per share                            8.87       49.68     4.18   
Total debt                                    19.06B       43.7B  121.64B   
Total debt/equity                             137.86      178.73   175.84   
Current ratio                                

In [310]:
# creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & amortisation",
         "Market cap (intra-day)",
         "Net income available to common shareholders",
         "Net cash provided by operating activities",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Net property, plant and equipment",
         "Total stockholders' equity",
         "Long-term debt",
         "Forward annual dividend yield"] # change as required

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]

In [311]:
all_stats = {}
for ticker in tickers:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)

can't read data for  MMM
can't read data for  AXP
can't read data for  AAPL
can't read data for  BA
can't read data for  CAT
can't read data for  CVX
can't read data for  CSCO
can't read data for  KO
can't read data for  DIS
can't read data for  DWDP
can't read data for  XOM
can't read data for  GE
can't read data for  GS
can't read data for  HD
can't read data for  IBM
can't read data for  INTC
can't read data for  JNJ
can't read data for  JPM
can't read data for  MCD
can't read data for  MRK
can't read data for  MSFT
can't read data for  NKE
can't read data for  PFE
can't read data for  PG
can't read data for  TRV
can't read data for  UTX
can't read data for  UNH
can't read data for  VZ
can't read data for  V
can't read data for  WMT


In [309]:
all_stats

{}